In [71]:
from selenium import webdriver
from bs4 import BeautifulSoup
from pyshadow.main import Shadow
from selenium.webdriver import ActionChains
from selenium.webdriver import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException            
from pynput.keyboard import Key, Controller
import keyboard
import os
import time
from WindowFinder import *
url = ""
import warnings
warnings.filterwarnings('ignore')

In [5]:
with open('senha.txt') as arquivo:
    linhas = arquivo.readlines()
    
LOGIN = linhas[0].strip()
SENHA = linhas[1].strip()

controller = Controller()

SITE_LOGIN = 'https://perfil.estrategia.com/login'
SITE_LOGIN_OK = 'https://perfil.estrategia.com/meu-perfil/'
SITE_CONTENT = 'https://www.estrategiaconcursos.com.br/app/dashboard/assinaturas'

XPATH_LOGIN = '//input[@name="loginField"]'
XPATH_PASSWORD = '//input[@name="passwordField"]'
XPATH_BTN_LOGIN = '//button[@type="submit"]'
XPATH_FECHA_NOTIFICACAO = '//a[@id="pushActionRefuse"]'
XPATH_SELECT_MATRICULAS = '//a[contains(@href, "app/dashboard/cursos/") or contains(@href, "app/dashboard/pacote")]'
XPATH_SELECT_CURSOS = '//a[contains(@href, "app/dashboard/cursos/")]'
XPATH_TITULO_CURSO = '//h2[@class="CourseInfo-content-title"]'
XPATH_LIST_AULAS = '//div[contains(@class, "LessonList-item")]'
XPATH_EXPAND_AULA = '//*[@class="Collapse-header"]'
XPATH_DOWNLOAD_SIMPLIFICADO = '//*[contains(text(),"versão simplificada")]'
XPATH_DOWNLOAD_COMPLETO = '//*[contains(text(),"versão original")]'
XPATH_DOWNLOAD_RESUMO = '//*[contains(text(),"Baixar Resumo")]'
XPATH_DOWNLOAD_SLIDE = '//*[contains(text(),"Baixar Slides")]'
XPATH_DOWNLOAD_MAPA = '//*[contains(text(),"Baixar Mapa Mental")]'

XPATH_DOWNLOAD_LIST = [
    XPATH_DOWNLOAD_COMPLETO,
    XPATH_DOWNLOAD_SIMPLIFICADO,
    XPATH_DOWNLOAD_RESUMO,
    XPATH_DOWNLOAD_SLIDE,
    XPATH_DOWNLOAD_MAPA
]

NOME_LIST = [
    'Completo',
    'Simples',
    'Resumo',
    'Slide',
    'Mapa'
]

PATH = os.getcwd() + "\\Download"

In [ ]:
win = WindowFinder()
win.find_window_wildcard(".*Salvar como*.") 

In [76]:


#Método para garantir que apenas a primeira aba esteja aberta com o conteúdo
def verifica_aba():
    for i in range(len(browser.window_handles)-1, 0, -1):
        browser.switch_to.window(browser.window_handles[i])
        time.sleep(0.5)
        browser.close()
        time.sleep(0.5)
    browser.switch_to.window(browser.window_handles[0])

def listar_matriculas():
    matriculas = browser.find_elements_by_xpath(XPATH_SELECT_MATRICULAS)

    for i in range(len(matriculas)):
        print(f'Cod: %s Curso: %s' % (i, matriculas[i].text))

    #opcao = int(input("Selecione a matrícula para entrar:"))
    opcao = 0
    matricula = matriculas[opcao].text
    clica(matriculas[opcao])
    time.sleep(5)
    return matricula
    
def listar_cursos():
    cursos = browser.find_elements_by_xpath(XPATH_SELECT_CURSOS)
    curso_lista = []
    for i in range(len(cursos)):
        #print(f'Cod: %s Curso: %s' % (i, cursos[i].text))
        curso_lista.append(cursos[i].text)

    time.sleep(1)

    #opcao = int(input("Selecione o curso para fazer o download:"))
    #opcao = 0
    #curso = cursos[opcao].text
    #a = cursos[0].get_property("")
    #clica(cursos[opcao])

    #while len(browser.find_elements_by_class_name('CourseInfo-content')) == 0:
    time.sleep(1)
    
    return curso_lista


def cria_pasta(path):
    try:
        os.makedirs(path)
    except FileExistsError:
        pass
    time.sleep(3)
#os.chdir('c:\\Users\\zehlu\\Documents\\GitHub Code\\EstudosIA\\Downloader Estratégia Concursos')

def clica(elemento):
    while True:    
        time.sleep(1)
        action.scroll_to_element(elemento).perform()
        time.sleep(0.75)
        if EC.element_to_be_clickable(elemento):
            time.sleep(0.5)
            browser.execute_script("arguments[0].click();", elemento)
            #elemento.click()
            break
'''
def clica(elemento):
    while not elemento.is_displayed:    
        action.move_to_element(elemento).perform()
        time.sleep(0.5)
    elemento.click()
'''
def clica_botao_direito(elemento):
    while True:
        action.scroll_to_element(elemento).perform()
        time.sleep(1)
        if EC.element_to_be_clickable(elemento):
            time.sleep(0.75)
            action.context_click(elemento).perform()
            break     
def seleciona_salvar_como():
    keyboard.send('down')
    keyboard.send('down')
    keyboard.send('down') 
    keyboard.send('down')
    keyboard.send('enter')

    while win.find_window_wildcard(".*Salvar como*.") == None:
        pass
    time.sleep(2)

def clica_salvar_como(elemento):
    clica_botao_direito(elemento)
    time.sleep(0.8)
    seleciona_salvar_como()

def save(curso, aula, xpath, nome):
    try:
        FOLDER = PATH + '\\' + curso +  '\\Aula ' + str(aula)
        cria_pasta(FOLDER)
        pdf = browser.find_element_by_xpath(xpath)
        clica_salvar_como(pdf)
        controller.type(FOLDER + '\\' + nome + '.pdf')
        time.sleep(1)
        keyboard.send('enter')
        time.sleep(1)
        return True

    except NoSuchElementException:
        print("Aula {} não tem {}".format(aula, nome))
        return True

    except Exception:
        return False


def abrir_estrategia():
    browser = webdriver.Chrome()
    action = ActionChains(browser)
    browser.get(SITE_LOGIN)
    time.sleep(3)
    return browser, action

def login():
    username = browser.find_element_by_xpath(XPATH_LOGIN) 
    password = browser.find_element_by_xpath(XPATH_PASSWORD)

    username.send_keys(LOGIN)
    password.send_keys(SENHA)

    time.sleep(2)
    clica(browser.find_element_by_xpath(XPATH_BTN_LOGIN))
    
    while browser.current_url != SITE_LOGIN_OK:
        time.sleep(1)
    content_open()

def content_open():
    while browser.current_url != SITE_CONTENT:
        browser.get(SITE_CONTENT)
        time.sleep(5)

    notic = browser.find_elements_by_xpath(XPATH_FECHA_NOTIFICACAO)
    if len(notic) > 0:
        clica(notic[0])

    shadow.set_explicit_wait(20, 5)
    a = shadow.find_elements('button[class="button primary"]')
    if len(a) > 0:
        clica(a[0])

def listar_cursos_pacote(browser):
    cursos = browser.find_elements_by_xpath(XPATH_SELECT_CURSOS)
    
    for i in range(len(cursos)):
        print(f'Cod: %s Curso: %s' % (i, cursos[i].text))
    time.sleep(1)
    
    opcao = int(input("Selecione o curso para fazer o download:"))
    browser.get(cursos[opcao].get_attribute("href"))
    time.sleep(5)

def baixar_aulas_curso(curso):
    a = browser.find_element_by_xpath('//*[contains(text(),"{}")]'.format(curso))
    clica(a)
    
    while len(browser.find_elements_by_xpath(XPATH_EXPAND_AULA)) == 0:
        time.sleep(5)

    aulas = browser.find_elements_by_xpath(XPATH_EXPAND_AULA)

    for i in range(len(aulas)):
        clica(aulas[i])
        for xpath, nome in zip(XPATH_DOWNLOAD_LIST, NOME_LIST):
            url = browser.current_url
            
            while not save(curso, i, xpath, nome):
                browser.get(url)
                time.sleep(10)

        
        clica(aulas[i]) 

In [7]:
browser, action = abrir_estrategia()
shadow = Shadow(browser)
login()


In [14]:
content_open()
listar_matriculas()
curso_lista = listar_cursos()

Cod: 0 Curso: 01. Receita Federal (Auditor Fiscal) Pacote Completo - 2022 (Pré-Edital)


In [77]:
for i in range(11, len(curso_lista)):
    content_open()
    listar_matriculas()
    baixar_aulas_curso(curso_lista[i])
    
#browser.back()
#StaleElementReferenceException            

Cod: 0 Curso: 01. Receita Federal (Auditor Fiscal) Pacote Completo - 2022 (Pré-Edital)
Aula 0 não tem Resumo
Aula 0 não tem Mapa
Aula 1 não tem Resumo
Aula 2 não tem Resumo
Aula 2 não tem Mapa
Aula 3 não tem Resumo
Aula 3 não tem Mapa
Aula 4 não tem Resumo
Aula 4 não tem Mapa
Aula 5 não tem Resumo
Aula 5 não tem Mapa
Aula 6 não tem Resumo
Aula 6 não tem Mapa
Aula 7 não tem Resumo
Aula 7 não tem Mapa
Aula 8 não tem Resumo
Aula 8 não tem Mapa
Aula 9 não tem Resumo
Aula 9 não tem Mapa
Aula 10 não tem Resumo
Aula 10 não tem Mapa
Aula 11 não tem Resumo
Aula 11 não tem Mapa
Aula 12 não tem Resumo
Aula 12 não tem Mapa
Aula 13 não tem Resumo
Aula 13 não tem Mapa
Aula 14 não tem Resumo
Aula 14 não tem Mapa
Aula 15 não tem Resumo
Aula 15 não tem Mapa
Aula 16 não tem Resumo
Aula 16 não tem Mapa
Aula 17 não tem Resumo
Aula 17 não tem Mapa
Aula 18 não tem Resumo
Aula 18 não tem Mapa
Aula 19 não tem Resumo
Aula 19 não tem Mapa
Aula 20 não tem Resumo
Aula 20 não tem Mapa
Aula 21 não tem Resumo
Aula 2